<a href="https://colab.research.google.com/github/AJamal27891/University-coursees-projects/blob/master/ConvBert_Matching_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download dependencies

In [2]:
!rm -r  test_trainer 

In [3]:
!pip install transformers
!pip install datasets

## load datasets 

In [4]:
dirty_walmart_amazon = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_walmart_amazon'
dirty_dblp_scholar = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_scholar'
dirty_amazon_ituns = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_amazon_itunes'
dirty_dblp_acm = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_acm'
abt_buy = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/abt_buy'
amazon_google = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/amazon_google'
QQP= r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/QQP'

## Test dataset merger functions

In [5]:
import pandas as pd 
data_A = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['samsung s5','iphone','xperia','sonny','hstc','xomi']})
data_B = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['iphone','xperia','sonny','huawi','xomi','samsung s5']})
lables = pd.DataFrame({'left_id':[1,2,3,4,5,6],'right_id':[6,1,2,3,4,5],'label':[1,1,1,1,0,1]})

In [14]:
total_data = lables.merge(data_A,left_on='left_id',right_on='id')
total_data = total_data.merge(data_B,left_on='right_id',right_on='id')
total_data

,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [9]:
import random
def augmentation(df):
  first = ['label']
  clist = df.columns.tolist()
  clist.remove(first[0])
  random.shuffle(clist)
  df1 = df[first+clist]
  random.shuffle(clist)
  df2 = df[first+clist]
  random.shuffle(clist)
  df3 = df[first+clist]
  random.shuffle(clist)
  df4 = df[first+clist]
  return df, df1,df2,df3,df4


def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  

def merger(data_A,data_B,train,test,valid,
           remove_ids=True,
           merge_attributes=True,
           drop_original=True,
           tuples=True):
  train_data = train.merge(data_A,left_on='ltable_id',right_on='id')
  train_data = train_data.merge(data_B,left_on='rtable_id',right_on='id')
  test_data = test.merge(data_A,left_on='ltable_id',right_on='id')
  test_data = test_data.merge(data_B,left_on='rtable_id',right_on='id')
  valid_data = valid.merge(data_A,left_on='ltable_id',right_on='id')
  valid_data = valid_data.merge(data_B,left_on='rtable_id',right_on='id')
  if remove_ids:
    train_data = remove_id_columns(train_data)
    test_data = remove_id_columns(test_data)
    valid_data = remove_id_columns(valid_data)
  if merge_attributes:
    dfs = augmentation(train_data)
    new_dfs = []
    for df in dfs:
      df.iloc[:,1:]  = train_data.iloc[:,1:].astype(str).apply(lambda x : ' [COL] '+x.name.strip('_y').strip('_x') +' [VAL] '+x)
      df['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x :" [CLS] " + ' '.join(x.astype(str)) ,axis=1)
      df['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :" [SEP] " + ' '.join(x.astype(str)) ,axis=1)
      new_dfs.append(df[['label','text_left','text_right']])
    train_data = pd.concat(new_dfs)
    
      
    test_data.iloc[:,1:]  = test_data.iloc[:,1:].astype(str).apply(lambda x : ' [COL] '+x.name.strip('_y').strip('_x') +' [VAL] '+x)
    test_data['text_left'] =test_data[[c for c in test_data.columns[1:] if c.endswith('_x')]].apply(lambda x :" [CLS] " + ' '.join(x.astype(str)) ,axis=1)
    test_data['text_right'] =test_data[[c for c in test_data.columns[1:] if c.endswith('_y')]].apply(lambda x :" [SEP] " + ' '.join(x.astype(str)) ,axis=1)
    valid_data.iloc[:,1:]  = valid_data.iloc[:,1:].astype(str).apply(lambda x : ' [COL]'+x.name.strip('_y').strip('_x') +' [VAL] '+x)
    valid_data['text_left'] =valid_data[[c for c in valid_data.columns[1:] if c.endswith('_x')]].apply(lambda x :" [CLS] " + ' '.join(x.astype(str)) ,axis=1)
    valid_data['text_right'] =valid_data[[c for c in valid_data.columns[1:] if c.endswith('_y')]].apply(lambda x :" [SEP] " + ' '.join(x.astype(str)) ,axis=1) 

  if drop_original:
    train_data= train_data[['label','text_left','text_right']]
    test_data= test_data[['label','text_left','text_right']]
    valid_data= valid_data[['label','text_left','text_right']]
  if tuples:
    train_data['tuples']= train_data[['text_left','text_right']].values.tolist()
    train_data['tuples'] = train_data['tuples'].apply(lambda x: ' '.join(x))
    test_data['tuples']= test_data[['text_left','text_right']].values.tolist()
    test_data['tuples'] = test_data['tuples'].apply(lambda x: ' '.join(x))
    valid_data['tuples']= valid_data[['text_left','text_right']].values.tolist()
    valid_data['tuples'] = valid_data['tuples'].apply(lambda x: ' '.join(x))

  return train_data, test_data, valid_data

# Data Preprocessing

## Amazon Google data

In [17]:
table_A = pd.read_csv(amazon_google+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(amazon_google+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(amazon_google+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(amazon_google+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text_left,text_right,tuples
0,0,[CLS] [COL] title [VAL] microsoft visio stan...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] microsoft visio stan...
1,0,[CLS] [COL] title [VAL] adobe premiere pro c...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe premiere pro c...
2,0,[CLS] [COL] title [VAL] adobe indesign cs3 u...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe indesign cs3 u...
3,0,[CLS] [COL] title [VAL] adobe creative suite...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe creative suite...
4,0,[CLS] [COL] title [VAL] adobe creative suite...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe creative suite...


In [18]:
AG_train

,label,text_left,text_right,tuples
0,0,[CLS] [COL] title [VAL] microsoft visio stan...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] microsoft visio stan...
1,0,[CLS] [COL] title [VAL] adobe premiere pro c...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe premiere pro c...
2,0,[CLS] [COL] title [VAL] adobe indesign cs3 u...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe indesign cs3 u...
3,0,[CLS] [COL] title [VAL] adobe creative suite...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe creative suite...
4,0,[CLS] [COL] title [VAL] adobe creative suite...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe creative suite...
...,...,...,...,...
6869,0,[CLS] [COL] title [VAL] x-oom internet movie...,[SEP] [COL] title [VAL] feral interactive th...,[CLS] [COL] title [VAL] x-oom internet movie...
6870,1,[CLS] [COL] title [VAL] pc hoyle jewels : sw...,[SEP] [COL] title [VAL] encore software 1066...,[CLS] [COL] title [VAL] pc hoyle jewels : sw...
6871,0,[CLS] [COL] title [VAL] autodesk discreet co...,[SEP] [COL] title [VAL] autodesk autodesk co...,[CLS] [COL] title [VAL] autodesk discreet co...
6872,0,[CLS] [COL] title [VAL] virtual labs : light...,[SEP] [COL] title [VAL] virtual labs : light...,[CLS] [COL] title [VAL] virtual labs : light...


In [19]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)


In [20]:
AG_train

,label,text_left,text_right,tuples
0,0,[CLS] [COL] title [VAL] adventure workshop p...,[SEP] [COL] title [VAL] encore software 1242...,[CLS] [COL] title [VAL] adventure workshop p...
1,0,[CLS] [COL] title [VAL] jumpstart 1st grade ...,[SEP] [COL] title [VAL] brain play 1st-3rd g...,[CLS] [COL] title [VAL] jumpstart 1st grade ...
2,1,[CLS] [COL] title [VAL] quicken 2007 home & ...,[SEP] [COL] title [VAL] quicken home and bus...,[CLS] [COL] title [VAL] quicken 2007 home & ...
3,0,[CLS] [COL] title [VAL] land before time : k...,[SEP] [COL] title [VAL] jumpstart kindergart...,[CLS] [COL] title [VAL] land before time : k...
4,1,[CLS] [COL] title [VAL] sibelius 5 professio...,[SEP] [COL] title [VAL] sibelius sibelius 5 ...,[CLS] [COL] title [VAL] sibelius 5 professio...
...,...,...,...,...
6985,1,[CLS] [COL] title [VAL] apple remote desktop...,[SEP] [COL] title [VAL] apple remote desktop...,[CLS] [COL] title [VAL] apple remote desktop...
6986,1,[CLS] [COL] title [VAL] boomerang webpage de...,[SEP] [COL] title [VAL] web page designer go...,[CLS] [COL] title [VAL] boomerang webpage de...
6987,1,[CLS] [COL] title [VAL] wisdom of the bible ...,[SEP] [COL] title [VAL] tri synergy inc wisd...,[CLS] [COL] title [VAL] wisdom of the bible ...
6988,0,[CLS] [COL] title [VAL] instant immersion ja...,[SEP] [COL] title [VAL] instant immersion it...,[CLS] [COL] title [VAL] instant immersion ja...


In [21]:
import os 
try:
  os.mkdir('merged_data')
  os.mkdir('merged_data/amazon_google')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/amazon_google/train.csv')
AG_test.to_csv(r'merged_data/amazon_google/test.csv')
AG_valid.to_csv(r'merged_data/amazon_google/valid.csv')



In [22]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/amazon_google/train.csv','merged_data/amazon_google/valid.csv'],
                                              'test':'merged_data/amazon_google/test.csv',
                                              })

Using custom data configuration default-3b43034c075bb555


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3b43034c075bb555/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("YituTech/conv-bert-base")

In [16]:
def tokenize_function(examples):
    # examples['text_left'].head()
    # examples['tuples'] = examples[["text_left","text_right"]].apply(lambda x: (x[0],x[1]),axis=1)
    return tokenizer(examples['tuples'], padding="max_length", truncation=True,max_length=128)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x7fac3065a170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [25]:
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenizer.decode(tokenized_datasets['train']['input_ids'][0])

'[CLS] [CLS] [ col ] title [ val ] adventure workshop preschool - 1st grade 8th edition [ col ] manufacturer [ val ] encore [ col ] price [ val ] 19. 99 [SEP] [ col ] title [ val ] encore software 12421 adventure workshop 8 1st - 3rd [ col ] manufacturer [ val ] nan [ col ] price [ val ] 17. 97 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [18]:
import torch
torch.cuda.empty_cache()

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("YituTech/conv-bert-base", num_labels=2)

Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",num_train_epochs=14,per_device_train_batch_size=16)

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [30]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ConvBertForSequenceClassification.forward` and have been ignored: text_right, text_left, tuples.
***** Running training *****
  Num examples = 7458
  Num Epochs = 14
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6538


Step,Training Loss
500,0.252600
1000,0.123500
1500,0.087700
2000,0.068000
2500,0.056400
3000,0.042300
3500,0.026300
4000,0.027400
4500,0.013500
5000,0.010300


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=6538, training_loss=0.05522880535963074, metrics={'train_runtime': 1918.8989, 'train_samples_per_second': 54.412, 'train_steps_per_second': 3.407, 'total_flos': 8521835467622400.0, 'train_loss': 0.05522880535963074, 'epoch': 14.0})

In [31]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `ConvBertForSequenceClassification.forward` and have been ignored: text_right, text_left, tuples.
***** Running Evaluation *****
  Num examples = 468
  Batch size = 8


{'eval_f1': 0.8954545454545455,
 'eval_loss': 0.9385485649108887,
 'eval_runtime': 3.0199,
 'eval_samples_per_second': 154.973,
 'eval_steps_per_second': 19.537}

## abt-BUy dataset

In [33]:
table_A = pd.read_csv(abt_buy+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(abt_buy+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(abt_buy+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(abt_buy+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(abt_buy+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text_left,text_right,tuples
0,0,[CLS] [COL] name [VAL] lg 24 ' lds4821ww sem...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' lds4821ww sem...
1,0,[CLS] [COL] name [VAL] lg 24 ' lds4821bb sem...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' lds4821bb sem...
2,1,[CLS] [COL] name [VAL] lg 24 ' ldf6920bb ful...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' ldf6920bb ful...
3,0,[CLS] [COL] name [VAL] lg 24 ' ldf6920ww ful...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' ldf6920ww ful...
4,0,[CLS] [COL] name [VAL] lg 24 ' ldf6920st ful...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' ldf6920st ful...


In [34]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)

In [35]:

import os 
try:
  os.mkdir('merged_data/abt_buy/')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/abt_buy/train.csv')
AG_test.to_csv(r'merged_data/abt_buy/test.csv')
AG_valid.to_csv(r'merged_data/abt_buy/valid.csv')


In [36]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/abt_buy/train.csv','merged_data/abt_buy/valid.csv'],
                                              'test':'merged_data/abt_buy/test.csv',
                                              })

Using custom data configuration default-d734a5cdf9f9c19f


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d734a5cdf9f9c19f/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [37]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 6572
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 412
    })
})

In [38]:
torch.cuda.empty_cache()
training_args = TrainingArguments("/content/drive/MyDrive/XLNet/abt-Buy",num_train_epochs=14,per_device_train_batch_size=16)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][1])
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ConvBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running training *****
  Num examples = 6572
  Num Epochs = 14
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5754


Step,Training Loss
500,0.187300
1000,0.177900
1500,0.162400
2000,0.158100
2500,0.155100
3000,0.151500
3500,0.157300
4000,0.145300
4500,0.136200
5000,0.136900


Saving model checkpoint to /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-500
Configuration saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-1000
Configuration saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-1500
Configuration saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-2000
Configuration saved in /content/drive/MyDrive/XLNet/abt-Buy/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/XLN

TrainOutput(global_step=5754, training_loss=0.15352084001071095, metrics={'train_runtime': 1766.7037, 'train_samples_per_second': 52.079, 'train_steps_per_second': 3.257, 'total_flos': 7509453297561600.0, 'train_loss': 0.15352084001071095, 'epoch': 14.0})

In [40]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()


The following columns in the evaluation set  don't have a corresponding argument in `ConvBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running Evaluation *****
  Num examples = 412
  Batch size = 8


{'eval_f1': 0.7533875338753389,
 'eval_loss': 1.3691786527633667,
 'eval_runtime': 2.6576,
 'eval_samples_per_second': 155.028,
 'eval_steps_per_second': 19.567}

## dirty_walmart_amazon

In [25]:
table_A = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text_left,text_right,tuples
0,0,[CLS] [COL] title [VAL] elite screens cinewh...,[SEP] [COL] title [VAL] cinegray ezframe ser...,[CLS] [COL] title [VAL] elite screens cinewh...
1,0,[CLS] [COL] title [VAL] elite screens cinewh...,[SEP] [COL] title [VAL] powergain prime visi...,[CLS] [COL] title [VAL] elite screens cinewh...
2,0,[CLS] [COL] title [VAL] elite screens cinegr...,[SEP] [COL] title [VAL] powergain prime visi...,[CLS] [COL] title [VAL] elite screens cinegr...
3,0,[CLS] [COL] title [VAL] elite screens cinewh...,[SEP] [COL] title [VAL] cinewhite prime visi...,[CLS] [COL] title [VAL] elite screens cinewh...
4,0,[CLS] [COL] title [VAL] elite screens cinegr...,[SEP] [COL] title [VAL] cinewhite prime visi...,[CLS] [COL] title [VAL] elite screens cinegr...


In [26]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)

In [27]:
import os 
try:
  os.mkdir('merged_data/dirty_walmart_amazon/')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/dirty_walmart_amazon/train.csv')
AG_test.to_csv(r'merged_data/dirty_walmart_amazon/test.csv')
AG_valid.to_csv(r'merged_data/dirty_walmart_amazon/valid.csv')


In [28]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/dirty_walmart_amazon/train.csv','merged_data/dirty_walmart_amazon/valid.csv'],
                                              'test':'merged_data/dirty_walmart_amazon/test.csv',
                                              })

Using custom data configuration default-bc972e8e60874d5d


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bc972e8e60874d5d/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [29]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 6146
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'token_type_ids', 'tuples'],
        num_rows: 386
    })
})

In [30]:
torch.cuda.empty_cache()
training_args = TrainingArguments("/content/drive/MyDrive/XLNet/dirty_walmart_amazon",num_train_epochs=14,per_device_train_batch_size=16)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [31]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][1])
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ConvBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running training *****
  Num examples = 6146
  Num Epochs = 14
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5390


Step,Training Loss
500,0.218800
1000,0.069600
1500,0.034800
2000,0.024700
2500,0.016700
3000,0.006600
3500,0.005600
4000,0.005000
4500,0.002300
5000,0.001700


Saving model checkpoint to /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-500
Configuration saved in /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-1000
Configuration saved in /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-1500
Configuration saved in /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/XLNet/dirty_walmart_amazon/checkpoint-2000
Configu

TrainOutput(global_step=5390, training_loss=0.03601146066122462, metrics={'train_runtime': 1641.8266, 'train_samples_per_second': 52.407, 'train_steps_per_second': 3.283, 'total_flos': 7022687152588800.0, 'train_loss': 0.03601146066122462, 'epoch': 14.0})

In [32]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `ConvBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running Evaluation *****
  Num examples = 386
  Batch size = 8


{'eval_f1': 0.9116022099447514,
 'eval_loss': 0.8740784525871277,
 'eval_runtime': 3.1101,
 'eval_samples_per_second': 124.113,
 'eval_steps_per_second': 15.755}